In [120]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import pathlib
import time
import os
import datetime
import ast

# turn response caching on and turn on logging to your terminal window
ox.config(log_console=True, use_cache=True)

ox.__version__

'1.0.1'

# Functions

In [81]:
def update(local_scores, global_scores):
    """
    PARAMETERS:
        local_scores: a dictionary where key is the node, and the value is the the score
        global_scores: same as current_scores
    OUTPUT:
        new_scores: a dictionary where key is the node, and the value is the the score
    """
    # update the scores of each node
    new_scores = {}
    for node in local_scores:
        if (node in global_scores): # check if dictionaries share key (or node)
            new_scores[node] = global_scores[node] # update the global betweenness score for each node
            
    return new_scores

def update_centrality_scores(stats_local_scores, stats_global_scores):
    """
    Updates centrality scores such as Betweenness, Closeness, and Degree centrality.
    """
    
    
    if isinstance(stats_global_scores, pd.DataFrame):
        # if it is a dataframe, convert into a series
        stats_global_scores = stats_global_scores.squeeze()
        
    # set the global scores
    global_btwn_scores = ast.literal_eval(stats_global_scores['betweenness_centrality'])
    global_closeness_scores = ast.literal_eval(stats_global_scores['closeness_centrality'])
    global_degree_scores = ast.literal_eval(stats_global_scores['degree_centrality'])
    
    for row in range(len(stats_local_scores)):
        try:
            # get the scores of each AGEB to be updated
            local_btwn = stats_local_scores['betweenness_centrality'].iloc[row]
            local_btwn_avg = stats_local_scores['betweenness_centrality_avg'].iloc[row]
            local_closeness = stats_local_scores['closeness_centrality'].iloc[row]
            local_closeness_avg = stats_local_scores['closeness_centrality_avg'].iloc[row]
            local_degree = stats_local_scores['degree_centrality'].iloc[row]
            local_degree_avg = stats_local_scores['degree_centrality_avg'].iloc[row]

           # using the ast modul to convert the string-like dict into a valid dict if not a valid dict
            if not isinstance(local_btwn, dict):
                local_btwn_scores = ast.literal_eval(local_btwn)
            if not isinstance(local_closeness, dict):
                local_closeness_scores = ast.literal_eval(local_closeness)
            if not isinstance(local_closeness, dict):
                local_degree_scores = ast.literal_eval(local_degree)
           
            local_btwn_scores = local_btwn
            local_closeness_scores = local_closeness
            local_degree_scores = local_degree

            # update the scores of each node
            new_btwn_scores = update(local_btwn_scores, global_btwn_scores)
            new_closeness_scores = update(local_closeness_scores, global_closeness_scores)
            new_degree_scores = update(local_degree_scores, global_degree_scores)

            # get the new average of each AGEB
            new_btwn_avg = sum(new_btwn_scores.values()) / len(new_btwn_scores)
            new_closeness_avg = sum(new_closeness_scores.values()) / len(new_closeness_scores)
            new_degree_avg = sum(new_degree_scores.values()) / len(new_degree_scores)

            # update into the dataframe
            local_btwn = new_btwn_scores
            local_btwn_avg = new_btwn_avg
            local_closeness = new_closeness_scores
            local_closeness_avg = new_closeness_avg
            local_degree = new_degree_scores
            local_degree_avg = new_degree_avg

        except Exception as e:
            print('Row {} failed: {}'.format(row, e))

In [6]:
def rename_stats_columns(stats_dataframe, inplace=False):
    """
    Rename column names for saving to shapefiles.
    """
    
    # columns to be changed
    cols = {'avg_neighbor_degree':'avg_n8d', 
              'avg_neighbor_degree_avg':'avg2_n8d',
              'avg_weighted_neighbor_degree':'avg_w_n8d',
              'avg_weighted_neighbor_degree_avg':'avg2_w_n8d',
              'betweenness_centrality':'btwn_c',
              'betweenness_centrality_avg':'btwn_c_avg',
              'circuity_avg':'cirq_avg',
              'clean_intersection_count':'cl_int_cnt',
              'clean_intersection_density_km':'cl_int_dkm',
              'closeness_centrality':'clos_c',
              'closeness_centrality_avg':'clos_c_avg',
              'clustering_coefficient':'c_cof',
              'clustering_coefficient_avg':'c_cof_avg',
              'clustering_coefficient_weighted':'c_cofw',
              'clustering_coefficient_weighted_avg':'c_cofw_avg',
              'degree_centrality':'degcen',
              'degree_centrality_avg':'degcen_avg',
              'eccentricity':'eccty',
              'edge_connectivity':'m_cvty',
              'edge_density_km':'m_dty_km',
              'edge_length_avg':'m_len_avg',
              'edge_length_total':'m_len_tot',
              'intersection_count':'int_cnt',
              'intersection_density_km':'int_dty_km',
              'node_connectivity':'n_cvty',
              'node_connectivity_avg':'n_cvty_avg',
              'node_density_km':'n_dty_km',
              'pagerank_max':'prnk_max',
              'pagerank_max_node':'prnk_max_n',
              'pagerank_min':'prnk_min',
              'pagerank_min_node':'prnk_min_n',
              'self_loop_proportion':'sloop_prop',
              'street_density_km':'st_dty_km',
              'street_length_avg':'st_len_avg',
              'street_length_total':'st_len_tot',
              'street_segments_count':'st_seg_cnt',
              'streets_per_node_avg':'st_n_avg',
              'streets_per_node_counts':'st_n_cnts',
              'streets_per_node_proportion':'st_n_prop'
             }
    
    if inplace is False:
        new_df = stats_dataframe
    
    if 'pop_density' in stats_dataframe.columns:
        if inplace is False:
            new_df.rename(columns={'pop_density':'popdensity'}, inplace=True)
        else:
            stats_dataframe.rename(columns={'pop_density':'popdensity'}, inplace=True)
            
    if inplace is False:
        new_df.rename(columns=cols, inplace=True)
        return new_df
    else:
        stats_dataframe.rename(columns=cols, inplace=True)

In [7]:
def save_shapefiles(geo_df, filename):
    # save data to shapefiles
    folder = './data/' + filename + '_shp'
    filepath = pathlib.Path(folder) / str(filename+'.shp')
    # if save folder does not already exist, create it
    filepath.parent.mkdir(parents=True, exist_ok=True)
    geo_df.to_file(filepath)

In [8]:
def change_census_data_type(dataframe):
    dataframe = dataframe.astype({
        'POBTOT':'float32',
        'POBFEM':'float32',
        'POBMAS':'float32',
        'P_0A2':'float32',
        'P_0A2_F':'float32',
        'P_0A2_M':'float32',
        'P_3YMAS':'float32',
        'P_3YMAS_F':'float32',
        'P_3YMAS_M':'float32',
        'P_5YMAS':'float32',
        'P_5YMAS_F':'float32',
        'P_5YMAS_M':'float32',
        'P_12YMAS':'float32',
        'P_12YMAS_F':'float32',
        'P_12YMAS_M':'float32',
        'P_15YMAS':'float32',
        'P_15YMAS_F':'float32',
        'P_15YMAS_M':'float32',
        'P_18YMAS':'float32',
        'P_18YMAS_F':'float32',
        'P_18YMAS_M':'float32',
        'P_3A5':'float32',
        'P_3A5_F':'float32',
        'P_3A5_M':'float32',
        'P_6A11':'float32',
        'P_6A11_F':'float32',
        'P_6A11_M':'float32',
        'P_8A14':'float32',
        'P_8A14_F':'float32',
        'P_8A14_M':'float32',
        'P_12A14':'float32',
        'P_12A14_F':'float32',
        'P_12A14_M':'float32',
        'P_15A17':'float32',
        'P_15A17_F':'float32',
        'P_15A17_M':'float32',
        'P_18A24':'float32',
        'P_18A24_F':'float32',
        'P_18A24_M':'float32',
        'P_15A49_F':'float32',
        'P_60YMAS':'float32',
        'P_60YMAS_F':'float32',
        'P_60YMAS_M':'float32',
        'REL_H_M':'float32',
        'POB0_14':'float32',
        'POB15_64':'float32',
        'POB65_MAS':'float32',
        'PROM_HNV':'float32',
        'PNACENT':'float32',
        'PNACENT_F':'float32',
        'PNACENT_M':'float32',
        'PNACOE':'float32',
        'PNACOE_F':'float32',
        'PNACOE_M':'float32',
        'PRES2015':'float32',
        'PRES2015_F':'float32',
        'PRES2015_M':'float32',
        'PRESOE15':'float32',
        'PRESOE15_F':'float32',
        'PRESOE15_M':'float32',
        'P3YM_HLI':'float32',
        'P3YM_HLI_F':'float32',
        'P3YM_HLI_M':'float32',
        'P3HLINHE':'float32',
        'P3HLINHE_F':'float32',
        'P3HLINHE_M':'float32',
        'P3HLI_HE':'float32',
        'P3HLI_HE_F':'float32',
        'P3HLI_HE_M':'float32',
        'P5_HLI':'float32',
        'P5_HLI_NHE':'float32',
        'P5_HLI_HE':'float32',
        'PHOG_IND':'float32',
        'POB_AFRO':'float32',
        'POB_AFRO_F':'float32',
        'POB_AFRO_M':'float32',
        'PCON_DISC':'float32',
        'PCDISC_MOT':'float32',
        'PCDISC_VIS':'float32',
        'PCDISC_LENG':'float32',
        'PCDISC_AUD':'float32',
        'PCDISC_MOT2':'float32',
        'PCDISC_MEN':'float32',
        'PCON_LIMI':'float32',
        'PCLIM_CSB':'float32',
        'PCLIM_VIS':'float32',
        'PCLIM_HACO':'float32',
        'PCLIM_OAUD':'float32',
        'PCLIM_MOT2':'float32',
        'PCLIM_RE_CO':'float32',
        'PCLIM_PMEN':'float32',
        'PSIND_LIM':'float32',
        'P3A5_NOA':'float32',
        'P3A5_NOA_F':'float32',
        'P3A5_NOA_M':'float32',
        'P6A11_NOA':'float32',
        'P6A11_NOAF':'float32',
        'P6A11_NOAM':'float32',
        'P12A14NOA':'float32',
        'P12A14NOAF':'float32',
        'P12A14NOAM':'float32',
        'P15A17A':'float32',
        'P15A17A_F':'float32',
        'P15A17A_M':'float32',
        'P18A24A':'float32',
        'P18A24A_F':'float32',
        'P18A24A_M':'float32',
        'P8A14AN':'float32',
        'P8A14AN_F':'float32',
        'P8A14AN_M':'float32',
        'P15YM_AN':'float32',
        'P15YM_AN_F':'float32',
        'P15YM_AN_M':'float32',
        'P15YM_SE':'float32',
        'P15YM_SE_F':'float32',
        'P15YM_SE_M':'float32',
        'P15PRI_IN':'float32',
        'P15PRI_INF':'float32',
        'P15PRI_INM':'float32',
        'P15PRI_CO':'float32',
        'P15PRI_COF':'float32',
        'P15PRI_COM':'float32',
        'P15SEC_IN':'float32',
        'P15SEC_INF':'float32',
        'P15SEC_INM':'float32',
        'P15SEC_CO':'float32',
        'P15SEC_COF':'float32',
        'P15SEC_COM':'float32',
        'P18YM_PB':'float32',
        'P18YM_PB_F':'float32',
        'P18YM_PB_M':'float32',
        'GRAPROES':'float32',
        'GRAPROES_F':'float32',
        'GRAPROES_M':'float32',
        'PEA':'float32',
        'PEA_F':'float32',
        'PEA_M':'float32',
        'PE_INAC':'float32',
        'PE_INAC_F':'float32',
        'PE_INAC_M':'float32',
        'POCUPADA':'float32',
        'POCUPADA_F':'float32',
        'POCUPADA_M':'float32',
        'PDESOCUP':'float32',
        'PDESOCUP_F':'float32',
        'PDESOCUP_M':'float32',
        'PSINDER':'float32',
        'PDER_SS':'float32',
        'PDER_IMSS':'float32',
        'PDER_ISTE':'float32',
        'PDER_ISTEE':'float32',
        'PAFIL_PDOM':'float32',
        'PDER_SEGP':'float32',
        'PDER_IMSSB':'float32',
        'PAFIL_IPRIV':'float32',
        'PAFIL_OTRAI':'float32',
        'P12YM_SOLT':'float32',
        'P12YM_CASA':'float32',
        'P12YM_SEPA':'float32',
        'PCATOLICA':'float32',
        'PRO_CRIEVA':'float32',
        'POTRAS_REL':'float32',
        'PSIN_RELIG':'float32',
        'TOTHOG':'float32',
        'HOGJEF_F':'float32',
        'HOGJEF_M':'float32',
        'POBHOG':'float32',
        'PHOGJEF_F':'float32',
        'PHOGJEF_M':'float32',
        'VIVTOT':'float32',
        'TVIVHAB':'float32',
        'TVIVPAR':'float32',
        'VIVPAR_HAB':'float32',
        'TVIVPARHAB':'float32',
        'VIVPAR_DES':'float32',
        'VIVPAR_UT':'float32',
        'OCUPVIVPAR':'float32',
        'PROM_OCUP':'float32',
        'PRO_OCUP_C':'float32',
        'VPH_PISODT':'float32',
        'VPH_PISOTI':'float32',
        'VPH_1DOR':'float32',
        'VPH_2YMASD':'float32',
        'VPH_1CUART':'float32',
        'VPH_2CUART':'float32',
        'VPH_3YMASC':'float32',
        'VPH_C_ELEC':'float32',
        'VPH_S_ELEC':'float32',
        'VPH_AGUADV':'float32',
        'VPH_AEASP':'float32',
        'VPH_AGUAFV':'float32',
        'VPH_TINACO':'float32',
        'VPH_CISTER':'float32',
        'VPH_EXCSA':'float32',
        'VPH_LETR':'float32',
        'VPH_DRENAJ':'float32',
        'VPH_NODREN':'float32',
        'VPH_C_SERV':'float32',
        'VPH_NDEAED':'float32',
        'VPH_DSADMA':'float32',
        'VPH_NDACMM':'float32',
        'VPH_SNBIEN':'float32',
        'VPH_REFRI':'float32',
        'VPH_LAVAD':'float32',
        'VPH_HMICRO':'float32',
        'VPH_AUTOM':'float32',
        'VPH_MOTO':'float32',
        'VPH_BICI':'float32',
        'VPH_RADIO':'float32',
        'VPH_TV':'float32',
        'VPH_PC':'float32',
        'VPH_TELEF':'float32',
        'VPH_CEL':'float32',
        'VPH_INTER':'float32',
        'VPH_STVP':'float32',
        'VPH_SPMVPI':'float32',
        'VPH_CVJ':'float32',
        'VPH_SINRTV':'float32',
        'VPH_SINLTC':'float32',
        'VPH_SINCINT':'float32',
        'VPH_SINTIC':'float32'
        })
    return dataframe

# Merida municipality road network

## Get graph with OSMnx

OSMnx lets you download street network data and build topologically-corrected street networks, project and plot the networks, and save the street network as SVGs, GraphML files, GeoPackages, or shapefiles for later use. The street networks are directed and preserve one-way directionality.

You can download a street network by providing OSMnx any of the following:
  - a bounding box
  - a lat-long point plus a distance
  - an address plus a distance
  - a place name or list of place names (to automatically geocode and get the boundary of)
  - a polygon of the desired street network's boundaries
  - a .osm formatted xml file
  
You can also specify several different network types:
  - 'drive' - get drivable public streets (but not service roads)
  - 'drive_service' - get drivable streets, including service roads
  - 'walk' - get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
  - 'bike' - get all streets and paths that cyclists can use
  - 'all' - download all non-private OSM streets and paths (this is the default network type unless you specify a different one)
  - 'all_private' - download all OSM streets and paths, including private-access ones

In the following, we define a function to download the graph for the city of Mérida, Yucatán, or, if already present, load it as an NetworkX graph object.

In [68]:
places = [{'county' : 'Merida',
           'state' : 'Yucatan',
           'country' : 'Mexico'}]

def get_roads_osmnx(places, update=False, proj=False, crs=None):

    dirpath = pathlib.Path('./networks/')
    filepath = dirpath/'merida-road.graphml'
    logpath = dirpath/'log'
                                    
    if filepath.exists() and not update:
        G = ox.load_graphml(filepath)
    else:
        # get drivable public streets network, aka road network, without service roads,
        # e.g. private, parking lots, etc.
        # use retain_all if you want to keep all disconnected subgraphs (e.g. when your places aren't adjacent)
        # TODO: It would be nice to setup up a polygon for the city and its surrounding areas, to be sure
        # exactly the location.
        G = ox.graph_from_place(places, network_type='drive')
        ox.save_graphml(G, filepath=filepath, gephi=False)
        
    if proj:
        G = ox.project_graph(G, to_crs=crs)
    
    print(f"Graph created at: {G.graph['created_date']}")
    return G, *ox.graph_to_gdfs(G)
        
#G, nodes, edges = get_roads_osmnx(places, update=False)
G_proj, nodes_proj, edges_proj = get_roads_osmnx(places, update=False, proj=True, crs=3857)

Graph created at: 2021-03-31 01:02:44


The Coordinate Reference System (CRS) is important because the geometric shapes in a GeoDataFrame object are simply a collection of coordinates in an arbitrary space. A CRS tells Python how those coordinates relate to places on the Earth. (Reference: https://geopandas.org/projections.html)

It is more convenient to work with a flattened 2D projection rather than its spherical coordinates. Usually this happens where we are interested in some region such as one city. The standard convention for such projected maps is to take $x =$ Easting and $y =$ Northing, in the order $(x, y)$, in meters from some origin. It is not possible to make a perfect flat version of an ellipsoid surface, so any projection must make some compromise. Usually a good projection to use is one of the Universal Transverse Mercator (UTM) family. UTM splits the Earth’s
surface into state-sized regions, and defines separate projection for each one, to minimize the distortion
there. (Retrieved from: https://link.springer.com/chapter/10.1007/978-3-319-72953-4_5)

The same CRS can often be referred to in many ways. For example, one of the most commonly used CRS is the WGS84 latitude-longitude projection. This can be referred to using the authority code "EPSG:4326". However, such EPSG is in degree units. For that reason, we will use an alternative option which is the "EPSG:3857" that is measured in meters. This projected coordinate system is the one that Google, OpenStreetMap, Bing, ArcGIS, ESRI, etc. use for rendering their maps. (See: https://epsg.io/3857)

In [73]:
G_proj.graph

{'created_date': '2021-03-31 01:02:44',
 'created_with': 'OSMnx 1.0.1',
 'crs': <Projected CRS: EPSG:3857>
 Name: WGS 84 / Pseudo-Mercator
 Axis Info [cartesian]:
 - X[east]: Easting (metre)
 - Y[north]: Northing (metre)
 Area of Use:
 - name: World between 85.06°S and 85.06°N.
 - bounds: (-180.0, -85.06, 180.0, 85.06)
 Coordinate Operation:
 - name: Popular Visualisation Pseudo-Mercator
 - method: Popular Visualisation Pseudo Mercator
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 'simplified': True}

In [4]:
print(f"The road network has {G_proj.number_of_edges()} edges and {G_proj.number_of_nodes()} nodes")

The road network has 93485 edges and 35105 nodes


In [5]:
## what sized area does our largest component cover in square meters?
graph_area_m = nodes_proj.unary_union.convex_hull.area
graph_area_km = graph_area_m / 1e6 # Squared meters? What is the oficial value?
graph_area_km # squared kilometers

1032.3653827209728

## Calculate stats

In [9]:
def get_stats(G):
    try:
        start_time = time.time()
        area_m = float(G.graph['area_m'])

        stats = ox.basic_stats(G, area=area_m, clean_intersects=True, circuity_dist='euclidean')
        
        # Convert MultiDiGraph to DiGraph.
        # Chooses between parallel edges by minimizing weight attribute value.
        D = ox.utils_graph.get_digraph(G, weight="length")
        # create undirected Graph from the DiGraph, for those metrics that need it
        Gu = nx.Graph(D)
        
        # average degree of the neighborhood of each node, and average for graph
        avg_neighbor_degree = nx.average_neighbor_degree(G)
        stats["avg_neighbor_degree"] = avg_neighbor_degree
        stats["avg_neighbor_degree_avg"] = sum(avg_neighbor_degree.values()) / len(avg_neighbor_degree)
        
        # avg weighted degree of neighborhood of each node, and average for graph
        avg_wtd_nbr_deg = nx.average_neighbor_degree(G, weight="length")
        stats["avg_weighted_neighbor_degree"] = avg_wtd_nbr_deg
        stats["avg_weighted_neighbor_degree_avg"] = sum(avg_wtd_nbr_deg.values()) / len(avg_wtd_nbr_deg)
        
        # degree centrality for a node is the fraction of nodes it is connected to
        degree_centrality = nx.degree_centrality(G)
        stats["degree_centrality"] = degree_centrality
        stats["degree_centrality_avg"] = sum(degree_centrality.values()) / len(degree_centrality)
        
        # calculate clustering coefficient for the nodes
        stats["clustering_coefficient"] = nx.clustering(Gu)
        
        # average clustering coefficient for the graph
        stats["clustering_coefficient_avg"] = nx.average_clustering(Gu)
        
        # calculate weighted clustering coefficient for the nodes
        stats["clustering_coefficient_weighted"] = nx.clustering(Gu, weight="length")
        
        # average clustering coefficient (weighted) for the graph
        stats["clustering_coefficient_weighted_avg"] = nx.average_clustering(Gu, weight="length")
        
        # pagerank: a ranking of the nodes in the graph based on the structure of
        # the incoming links
        pagerank = nx.pagerank(D, weight="length")
        stats["pagerank"] = pagerank
        
        # node with the highest page rank, and its value
        pagerank_max_node = max(pagerank, key=lambda x: pagerank[x])
        stats["pagerank_max_node"] = pagerank_max_node
        stats["pagerank_max"] = pagerank[pagerank_max_node]
        
        # node with the lowest page rank, and its value
        pagerank_min_node = min(pagerank, key=lambda x: pagerank[x])
        stats["pagerank_min_node"] = pagerank_min_node
        stats["pagerank_min"] = pagerank[pagerank_min_node]
        
        # betweenness centrality of a node is the sum of the fraction of
        # all-pairs shortest paths that pass through node. nx2.4+
        # implementation cannot run on Multi(Di)Graphs, so use DiGraph
        btwn_cent = nx.betweenness_centrality(D, weight="length")
        stats["betweenness_centrality"] = btwn_cent
        stats["betweenness_centrality_avg"] = sum(btwn_cent.values()) / len(btwn_cent)
        
        # closeness centrality of a node is the reciprocal of the sum of the
        # shortest path distances from u to all other nodes
        close_cent = nx.closeness_centrality(G, distance="length")
        stats["closeness_centrality"] = close_cent
        stats["closeness_centrality_avg"] = sum(close_cent.values()) / len(close_cent)
        
        # extract nodes of the graph
        node_list = [node for node in close_cent]
        stats['nodes'] = node_list
        
        stats['area_km'] = area_m / 1e6
        stats['area'] = area_m
        stats['time'] = time.time()-start_time
        return pd.Series(stats)

    except Exception as e:
        print('failed: {}'.format(e))
        return pd.Series()

In [10]:
G_proj.graph['area_m'] = graph_area_m

In [13]:
%%time
road_network_stats = get_stats(G_proj)

CPU times: user 6h 27min 54s, sys: 16.1 s, total: 6h 28min 10s
Wall time: 6h 29min 6s


For the road network of the whole city we are not counting measures for connectivity, eccentricity, diameter, radius, center, and periphery due to memory overflow.

Save stats in CSV:

In [14]:
stats_road_network = road_network_stats.to_frame()
stats_road_network = stats_road_network.T

# save stats in a csv for further usage to not recalculate stats again
stats_road_network.to_csv('./data/stats_merida_road_network.csv', encoding='utf-8', index=False)

## Load stats

In [118]:
stats_road_network = pd.read_csv('./data/stats_merida_road_network.csv', encoding='utf-8')

# AGEBs road network

## Prepare data

Read data from INEGI's Marco Geoestadístico - Censo de Poblacion y Vivienda 2020 (https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=889463807469)

In order to perform some analysis involving how data metrics change with scale, we import the AGEB (Areas Geoestadísticas Básicas Urbanas) of the city from the Marco Geoestadístico and Censo 2020 to create a subgraph for each different AGEB.

From the data from the Marco Geoestadistico we are only using the geometries because it does not contain information from Censo 2020.

Geometries we can use:

<ul>
    <li>Areas geoestadísticas básicas urbanas (AGEB). Filename: ee<strong>a</strong></li>
    <li>Areas geoestadísticas básicas rurales (AGEB). Filename: ee<strong>ar</strong></li>
</ul>

Note: **ee** refers to the numeric key of the federal state: 01, 02,...,32

Unfortunately, INEGI only census urban AGEBs. For such reason, we are using Yucatan AGEB info: 31a

In [9]:
yucatan_ageb_marcogeo2020 = gpd.read_file('./input_data/marco_geoestadistico_2020/31_yucatan/conjunto_de_datos/31a.shp')
yucatan_ageb_marcogeo2020.head()

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,3100100010130,31,001,0001,0130,"POLYGON ((3776847.045 1015576.473, 3776850.565..."
1,3100100010145,31,001,0001,0145,"POLYGON ((3776745.860 1015499.562, 3776745.439..."
2,3100100010164,31,001,0001,0164,"POLYGON ((3776136.004 1014614.653, 3776022.228..."
3,3100100010126,31,001,0001,0126,"POLYGON ((3776247.740 1014594.755, 3776229.182..."
4,3100100010107,31,001,0001,0107,"POLYGON ((3776613.825 1015270.341, 3776615.972..."


In [10]:
print("Number of total Urban AGEBs: {}".format(len(yucatan_ageb_marcogeo2020)))

Number of total Urban AGEBs: 1532


Read data from Censo de Población y Vivienda 2020 > Principales resultados por AGEB y manzana urbana (https://www.inegi.org.mx/programas/ccpv/2020/#Datos_abiertos) 

For Yucatan:(https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_31_cpv2020_csv.zip)

In [11]:
yucatan_ageb_censo2020 = pd.read_csv('./input_data/ageb_mza_urbana_31_cpv2020_csv/conjunto_de_datos/conjunto_de_datos_ageb_urbana_31_cpv2020.csv')
yucatan_ageb_censo2020.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,31,Yucatán,0,Total de la entidad Yucatán,0,Total de la entidad,0000,0,2320898,1180619,...,176401,580440,339142,306523,129272,60161,33887,67719,283586,16519
1,31,Yucatán,1,Abalá,0,Total del municipio,0000,0,6550,3247,...,6,1383,345,542,36,15,137,436,1394,78
2,31,Yucatán,1,Abalá,7,Total de la localidad urbana,0000,0,2608,1314,...,3,467,58,38,4,*,54,201,599,32
3,31,Yucatán,1,Abalá,7,Total AGEB urbana,0018,0,1336,679,...,0,227,15,13,*,*,28,112,317,16
4,31,Yucatán,1,Abalá,7,Uayalceh,0018,1,62,34,...,0,15,3,*,*,0,0,6,17,0


In [12]:
#Replace with a 0 the data values that do not have a record
yucatan_ageb_censo2020 = yucatan_ageb_censo2020.replace('*', '0')
yucatan_ageb_censo2020 = yucatan_ageb_censo2020.replace('N/D', '0')

In [13]:
# change data types from string to float32
yucatan_ageb_censo2020 = change_census_data_type(yucatan_ageb_censo2020)

In [14]:
# Select only the totals; the value 0 in the MZA columns represent the totals
yucatan_censo_totales = yucatan_ageb_censo2020[yucatan_ageb_censo2020['MZA'] == 0]

# Select the totals of each AGEB and drop the totals of municipalities and localities
yucatan_ageb_totales_censo2020 = yucatan_censo_totales.drop(yucatan_censo_totales[yucatan_censo_totales['AGEB'] == '0000'].index, inplace = False)

yucatan_ageb_totales_censo2020.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
3,31,Yucatán,1,Abalá,7,Total AGEB urbana,0018,0,1336.0,679.0,...,0.0,227.0,15.0,13.0,0.0,0.0,28.0,112.0,317.0,16.0
31,31,Yucatán,1,Abalá,7,Total AGEB urbana,0022,0,1272.0,635.0,...,3.0,240.0,43.0,25.0,3.0,0.0,26.0,89.0,282.0,16.0
59,31,Yucatán,2,Acanceh,1,Total AGEB urbana,0034,0,4873.0,2458.0,...,101.0,1073.0,572.0,444.0,70.0,29.0,41.0,152.0,576.0,26.0
106,31,Yucatán,2,Acanceh,1,Total AGEB urbana,0072,0,394.0,195.0,...,4.0,99.0,38.0,29.0,3.0,0.0,4.0,10.0,58.0,0.0
115,31,Yucatán,2,Acanceh,1,Total AGEB urbana,0087,0,2846.0,1425.0,...,60.0,604.0,247.0,277.0,32.0,26.0,32.0,92.0,401.0,20.0


In order to merge the data from the Marco Geoestadístico and the Census 2020, we need to create a new column where we took values from ENTIDAD, MUN, LOC, and AGEB to conform the CVEGEO (Clave Geoestadistica Concatenada) in the Census datafrane

The CVEGEO has a length of 13 characters if we don't count the MZAs.

Check the lengths of the elements that conform the CVEGEO:

In [121]:
# Check the length of the clave geoestadistica that do not count the manzanas
print(f"Clave Geoestadistica: {len(yucatan_ageb_marcogeo2020['CVEGEO'].iloc[0])}")

# Entidad
print(f"Entidad: {len(yucatan_ageb_marcogeo2020['CVE_ENT'].iloc[0])}")

# Municipio
print(f"Municipio: {len(yucatan_ageb_marcogeo2020['CVE_MUN'].iloc[0])}")

# Localidad
print(f"Localidad: {len(yucatan_ageb_marcogeo2020['CVE_LOC'].iloc[0])}")

# AGEB
print(f"AGEB: {len(yucatan_ageb_marcogeo2020['CVE_AGEB'].iloc[0])}")

Clave Geoestadistica: 13
Entidad: 2
Municipio: 3
Localidad: 4
AGEB: 4


In [17]:
# We need to add missing zeros to the values on the elements from the CENSO 2020 dataframe that conform the CVEGEO.
yucatan_ageb_totales_censo2020['MUN'] = yucatan_ageb_totales_censo2020['MUN'].apply(lambda x: '{0:0>3}'.format(x))
yucatan_ageb_totales_censo2020['LOC'] = yucatan_ageb_totales_censo2020['LOC'].apply(lambda x: '{0:0>4}'.format(x))

# Change data type from int to string so we can manipulate data for merging on the CVEGEO
yucatan_ageb_totales_censo2020['ENTIDAD'] = yucatan_ageb_totales_censo2020['ENTIDAD'].astype('str')
yucatan_ageb_totales_censo2020['MUN'] = yucatan_ageb_totales_censo2020['MUN'].astype('str')
yucatan_ageb_totales_censo2020['LOC'] = yucatan_ageb_totales_censo2020['LOC'].astype('str')

# Concatenate columns to create the CVEGEO column
yucatan_ageb_totales_censo2020['CVEGEO'] = yucatan_ageb_totales_censo2020['ENTIDAD'] + yucatan_ageb_totales_censo2020['MUN'] + yucatan_ageb_totales_censo2020['LOC'] + yucatan_ageb_totales_censo2020['AGEB']
yucatan_ageb_totales_censo2020.reset_index(inplace=True, drop=True)
yucatan_ageb_totales_censo2020.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,CVEGEO
0,31,Yucatán,001,Abalá,0007,Total AGEB urbana,0018,0,1336.0,679.0,...,227.0,15.0,13.0,0.0,0.0,28.0,112.0,317.0,16.0,3100100070018
1,31,Yucatán,001,Abalá,0007,Total AGEB urbana,0022,0,1272.0,635.0,...,240.0,43.0,25.0,3.0,0.0,26.0,89.0,282.0,16.0,3100100070022
2,31,Yucatán,002,Acanceh,0001,Total AGEB urbana,0034,0,4873.0,2458.0,...,1073.0,572.0,444.0,70.0,29.0,41.0,152.0,576.0,26.0,3100200010034
3,31,Yucatán,002,Acanceh,0001,Total AGEB urbana,0072,0,394.0,195.0,...,99.0,38.0,29.0,3.0,0.0,4.0,10.0,58.0,0.0,3100200010072
4,31,Yucatán,002,Acanceh,0001,Total AGEB urbana,0087,0,2846.0,1425.0,...,604.0,247.0,277.0,32.0,26.0,32.0,92.0,401.0,20.0,3100200010087


Now we need to merge both dataframes

In [18]:
#drop columns that are duplicated in the other dataframe
yucatan_ageb_totales_censo2020.drop(columns=['ENTIDAD', 'MUN', 'LOC', 'AGEB', 'MZA'], inplace = True)

yucatan_ageb = yucatan_ageb_marcogeo2020.merge(yucatan_ageb_totales_censo2020, on=['CVEGEO'], how='inner')
yucatan_ageb.head()

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,3100100070018,31,001,0007,0018,"POLYGON ((3785064.186 1021028.692, 3785092.159...",Yucatán,Abalá,Total AGEB urbana,1336.0,...,0.0,227.0,15.0,13.0,0.0,0.0,28.0,112.0,317.0,16.0
1,3100100070022,31,001,0007,0022,"POLYGON ((3785061.885 1021029.255, 3785064.186...",Yucatán,Abalá,Total AGEB urbana,1272.0,...,3.0,240.0,43.0,25.0,3.0,0.0,26.0,89.0,282.0,16.0
2,3100200010091,31,002,0001,0091,"POLYGON ((3800180.887 1034593.102, 3800170.226...",Yucatán,Acanceh,Total AGEB urbana,3804.0,...,51.0,796.0,159.0,427.0,42.0,23.0,36.0,115.0,652.0,22.0
3,3100200010087,31,002,0001,0087,"POLYGON ((3799738.530 1036119.757, 3799738.505...",Yucatán,Acanceh,Total AGEB urbana,2846.0,...,60.0,604.0,247.0,277.0,32.0,26.0,32.0,92.0,401.0,20.0
4,3100200010034,31,002,0001,0034,"POLYGON ((3797608.141 1036097.159, 3797608.517...",Yucatán,Acanceh,Total AGEB urbana,4873.0,...,101.0,1073.0,572.0,444.0,70.0,29.0,41.0,152.0,576.0,26.0


In [19]:
print("Number of total Census AGEBs: {}".format(len(yucatan_ageb_totales_censo2020)))
print("Number of AGEBs after merging: {}".format(len(yucatan_ageb)))

Number of total Census AGEBs: 1432
Number of AGEBs after merging: 1432


The Marco Geoestadistico has geometric data from 1532 AGEBS, while the Censo 2020 has census data from 1432 AGEBS. At the end, the merge of both dataframes result in 1432 AGEBs with its geometries. Recall that INEGI only census urban AGEBs.

As we are working only with the city of Merida, we need to select only AGEBs from this municipality:

In [20]:
#query to filter only the agebs from Merida municipality of Yucatan state
merida_ageb = yucatan_ageb[yucatan_ageb['NOM_MUN'] == 'Mérida']

merida_ageb.reset_index(inplace=True, drop=True)
merida_ageb.head()

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry,NOM_ENT,NOM_MUN,NOM_LOC,POBTOT,...,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,3105000015329,31,050,0001,5329,"POLYGON ((3779478.623 1047307.492, 3779480.973...",Yucatán,Mérida,Total AGEB urbana,2199.0,...,308.0,583.0,400.0,270.0,116.0,54.0,14.0,40.0,222.0,4.0
1,3105000012466,31,050,0001,2466,"POLYGON ((3777875.435 1056949.722, 3777947.632...",Yucatán,Mérida,Total AGEB urbana,3049.0,...,488.0,853.0,684.0,565.0,329.0,177.0,11.0,9.0,165.0,0.0
2,310500001249A,31,050,0001,249A,"POLYGON ((3779713.782 1053802.753, 3779711.800...",Yucatán,Mérida,Total AGEB urbana,1208.0,...,317.0,442.0,413.0,288.0,270.0,116.0,10.0,0.0,29.0,0.0
3,3105000015827,31,050,0001,5827,"POLYGON ((3772091.762 1052572.146, 3772125.684...",Yucatán,Mérida,Total AGEB urbana,2032.0,...,302.0,693.0,552.0,352.0,288.0,149.0,14.0,8.0,98.0,0.0
4,3105000014706,31,050,0001,4706,"POLYGON ((3772829.128 1047931.320, 3772821.774...",Yucatán,Mérida,Total AGEB urbana,3222.0,...,303.0,888.0,631.0,269.0,215.0,82.0,12.0,11.0,233.0,0.0


In [122]:
print("Merida has {} urban AGEB". format(len(merida_ageb)))

Merida has 523 urban AGEB


In [22]:
#view info of the Coordinate Reference System (CRS)
merida_ageb.crs

<Projected CRS: PROJCS["MEXICO_ITRF_2008_LCC",GEOGCS["ITRF2008",DA ...>
Name: MEXICO_ITRF_2008_LCC
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: International Terrestrial Reference Frame 2008
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [23]:
# Re-project to the same CRS of the projected graph for units in meters
merida_ageb = merida_ageb.to_crs(epsg=3857)
merida_ageb.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
# save geodataframes to shapefiles
save_shapefiles(merida_ageb, 'merida_ageb_census_data')

<ipython-input-7-f27acff0a9bb>:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo_df.to_file(filepath)


## Get every AGEB graph

In [26]:
gdf = gpd.read_file('./data/merida_ageb_census_data_shp/merida_ageb_census_data.shp')

In [27]:
# get area of each AGEB, in meters (as it is already in a CRS with meter unit)
def get_area(geometry):
    return geometry.area

gdf['area_m'] = gdf['geometry'].map(get_area)

In [28]:
# project it from CRS 3857 to 4326 for OSM projection
gdf = gdf.to_crs(epsg=4326)
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [29]:
# where to save networks
gdf['ENT_folder'] = gdf.apply(lambda row: '{}_{}'.format(row['CVE_ENT'], row['NOM_ENT']).replace(' ', '-'), axis=1)
gdf['MUN_folder'] = gdf.apply(lambda row: '{}_{}'.format(row['CVE_MUN'], row['NOM_MUN']).replace(' ', '-'), axis=1)
gdf['LOC_folder'] = gdf.apply(lambda row: '{}_{}'.format(row['CVE_LOC'], row['NOM_LOC']).replace(' ', '-'), axis=1)
gdf['AGEB_folder'] = gdf.apply(lambda row: '{}/{}/{}/{}'.format(row['ENT_folder'], 
                                                              row['MUN_folder'],
                                                             row['LOC_folder'],
                                                              row['CVE_AGEB'].replace(' ', '-')), axis=1)

In [30]:
# create list of queries
queries = gdf.apply(lambda row: {'AGEB_folder':row['AGEB_folder'], 
                                 'geometry':row['geometry'],
                                 'area_m':row['area_m'],
                                 'POBTOT':row['POBTOT']}, axis=1).tolist()
queries[0]

{'AGEB_folder': '31_Yucatán/050_Mérida/0001_Total-AGEB-urbana/5329',
 'geometry': <shapely.geometry.polygon.Polygon at 0x7fdf1c31ce20>,
 'area_m': 391488.2052492691,
 'POBTOT': 2199.0}

In [32]:
output_folder = './data/merida_ageb_graphs/' #where to save graph shapefiles and graphml

In [33]:
network_type = 'drive'
retain_all = True
buffer = False

In [35]:
%%time
start_time = time.time()
for query in queries:
    try:
        # load graph and save it if it hasn't already been saved in the output_path
        if not os.path.exists('{}/{}.graphml'.format(output_folder, query['AGEB_folder'])):
            geometry = query['geometry'].buffer(0) #fix trivially invalid geometries (nested shells, ring self-intersections)
            G_ageb = ox.graph_from_polygon(polygon=geometry, network_type=network_type, retain_all=retain_all)
            G_ageb.graph['name'] = query['AGEB_folder']
            G_ageb.graph['area_m'] = query['area_m']
            G_ageb.graph['POBTOT'] = query['POBTOT']
            ox.save_graph_shapefile(G_ageb, filepath=output_folder+query['AGEB_folder'], directed=True)
            ox.save_graphml(G_ageb, filepath=output_folder+'{}.graphml'.format(query['AGEB_folder']))
    except Exception as e:
        print('"{}" failed: {}'.format(query['AGEB_folder'], e))
print('Finished making graphs in {:,.2f} seconds'.format(time.time()-start_time))

"31_Yucatán/050_Mérida/0001_Total-AGEB-urbana/6204" failed: Found no graph nodes within the requested polygon
"31_Yucatán/050_Mérida/0084_Total-AGEB-urbana/6308" failed: Graph has no edges, cannot convert to a GeoDataFrame.
"31_Yucatán/050_Mérida/0075_Total-AGEB-urbana/4424" failed: Graph has no edges, cannot convert to a GeoDataFrame.
"31_Yucatán/050_Mérida/0093_Total-AGEB-urbana/3820" failed: Graph has no edges, cannot convert to a GeoDataFrame.
"31_Yucatán/050_Mérida/0075_Total-AGEB-urbana/6026" failed: Graph has no edges, cannot convert to a GeoDataFrame.
"31_Yucatán/050_Mérida/0001_Total-AGEB-urbana/6350" failed: Found no graph nodes within the requested polygon
Finished making graphs in 1,903.32 seconds
CPU times: user 10min 2s, sys: 11.6 s, total: 10min 14s
Wall time: 31min 43s


## Calculate stats

Make a dataframe with the filepath to every AGEB graphml

In [38]:
data_folder = './data/merida_ageb_graphs' # choose the folder where the saved graphs are located

In [39]:
places = []
for state_folder in os.listdir(data_folder):
    for city_folder in os.listdir('{}/{}'.format(data_folder, state_folder)):
        for town_folder in os.listdir('{}/{}/{}'.format(data_folder, state_folder, city_folder)):
            for ageb_file in os.listdir('{}/{}/{}/{}'.format(data_folder, state_folder, city_folder, town_folder)):
                if '.graphml' in ageb_file:
                    data = {}
                    data['CVE_ENT'] = state_folder.split('_')[0]
                    data['NOM_ENT'] = state_folder.split('_')[1]
                    data['CVE_MUN'] = city_folder.split('_')[0]
                    data['NOM_MUN'] = city_folder.split('_')[1]
                    data['CVE_LOC'] = town_folder.split('_')[0]
                    data['NOM_LOC'] = town_folder.replace('{}_'.format(data['CVE_LOC']), '').replace('-', ' ')
                    data['AGEB'] = ageb_file.replace('.graphml', '').replace('-', '')
                    data['CVEGEO'] = data['CVE_ENT']+data['CVE_MUN']+data['CVE_LOC']+data['AGEB']
                    data['path'] = '{}/{}/{}/{}'.format(data_folder, state_folder, city_folder, town_folder)
                    data['file'] = ageb_file
                    places.append(data)

df = pd.DataFrame(places)
df.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AGEB,CVEGEO,path,file
0,31,Yucatán,050,Mérida,0093,Total AGEB urbana,3816,3105000933816,./data/merida_ageb_graphs/31_Yucatán/050_Mérid...,3816.graphml
1,31,Yucatán,050,Mérida,0093,Total AGEB urbana,5668,3105000935668,./data/merida_ageb_graphs/31_Yucatán/050_Mérid...,5668.graphml
2,31,Yucatán,050,Mérida,0093,Total AGEB urbana,0135,3105000930135,./data/merida_ageb_graphs/31_Yucatán/050_Mérid...,0135.graphml
3,31,Yucatán,050,Mérida,0093,Total AGEB urbana,5653,3105000935653,./data/merida_ageb_graphs/31_Yucatán/050_Mérid...,5653.graphml
4,31,Yucatán,050,Mérida,0093,Total AGEB urbana,2127,3105000932127,./data/merida_ageb_graphs/31_Yucatán/050_Mérid...,2127.graphml


Load each graph and calculate stats

In [43]:
def load_graph_get_stats(row):
    
    try:
        start_time = time.time()
        G = ox.load_graphml(filepath=row['path']+'/'+row['file'])
        G = ox.project_graph(G, to_crs=3857) # project graph to CRS pseudo-UTM with meter units
        area_m = float(G.graph['area_m'])
        
        stats = ox.basic_stats(G, area=area_m, clean_intersects=True, circuity_dist='euclidean')
        stats['CVEGEO'] = row['CVEGEO']
        stats['CVE_ENT'] = row['CVE_ENT']
        stats['NOM_ENT'] = row['NOM_ENT']
        stats['CVE_MUN'] = row['CVE_MUN']
        stats['NOM_MUN'] = row['NOM_MUN']
        stats['CVE_LOC'] = row['CVE_LOC']
        stats['NOM_LOC'] = row['NOM_LOC']
        stats['AGEB'] = row['AGEB']
        
        # calculate extended stats
        extended_stats = ox.extended_stats(G, connectivity=True, anc=True, ecc=True, bc=True, cc=True)
        se = pd.Series(extended_stats)
        
        extended_stats_clean = se.to_dict()
        
        for key in extended_stats_clean:
            stats[key] = extended_stats_clean[key]
        
        # extract nodes of the graph
        node_list = [node for node in stats['closeness_centrality']]
        stats['nodes'] = node_list
            
        stats['area_km'] = area_m / 1e6
        stats['area'] = area_m
        stats['time'] = time.time()-start_time
        return pd.Series(stats)

    except Exception as e:
        print('{} at index {} failed: {}'.format(row['CVEGEO'], row.name, e))
        return pd.Series()

In [44]:
%%time
stats_ageb = df.apply(load_graph_get_stats, axis=1)

310500001085A at index 327 failed: source and sink are the same node


<ipython-input-43-ec54872e8202>:39: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()


3105000754439 at index 514 failed: source and sink are the same node


<ipython-input-43-ec54872e8202>:39: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()


CPU times: user 54min 44s, sys: 12.2 s, total: 54min 56s
Wall time: 55min 30s


There were two AGEB with CVEGEO 310500001085A, and 3105000754439 that failed. However, they were set as NaN in the dataframe. Let's drop them from the dataframe:

In [46]:
stats_ageb.drop(stats_ageb.index[[327, 514]], inplace=True)
stats_ageb.reset_index(inplace=True, drop=True)

In [47]:
len(stats_ageb)

515

## Update centrality scores

In [82]:
update_centrality_scores(stats_ageb, stats_road_network)

Row 170 failed: division by zero
Row 197 failed: division by zero


In [91]:
stats_ageb.drop(stats_ageb.index[[170, 197]], inplace=True)
stats_ageb.reset_index(inplace=True, drop=True)

In [92]:
len(stats_ageb)

513

After getting the graphs and calculating the stats, we ended up having 513 AGEB available to use in this work.

In [93]:
# save stats in a csv for further usage to not recalculate stats again
stats_ageb.to_csv('./data/stats_merida_ageb.csv', encoding='utf-8', index=False)

## Merge stats and census data

In [107]:
stats_ageb = pd.read_csv('./data/stats_merida_ageb.csv', encoding='utf-8')

In [108]:
census_ageb = gpd.read_file('./data/merida_ageb_census_data_shp/merida_ageb_census_data.shp')

In [109]:
# drop duplicate columns 
stats_ageb.drop(columns=['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AGEB'], inplace=True)

In [110]:
# merge both dataframes on CVEGEO
stats_census_ageb = census_ageb.merge(stats_ageb, on=['CVEGEO'], how='inner')

In [111]:
# add new column for population density
stats_census_ageb['popdensity'] = stats_census_ageb['POBTOT'] / stats_census_ageb['area_km']

## Identify AGEB by Primary Zone

Primary Zones are defined in the Mérida's Municipal Urban Development Plan (PMDU) (see: http://isla.merida.gob.mx/ServiciosInternet/OrdenamientoTerritorial/docs/PMDU.PDF, and http://isla.merida.gob.mx/serviciosinternet/ordenamientoterritorial/paginas/pmdu.phpx).

These primary zones are:
<ol>
    <li>Urban Consolidation (ZCO)</li>
    <li>Urban growth (ZCR)</li>
    <li>Regeneration and Sustainable Development (ZRS)</li>
    <li>Conservation of Natural Resources (ZRN)</li>
</ol>

In [119]:
primary_zones = gpd.read_file('./input_data/PMDU/ZONAS PRIMARIAS/ZONAS PRIMARIAS.shp')

In [113]:
# project the primary zones geodataframe to the same as we use in this work
primary_zones = primary_zones.to_crs(epsg=3857)

### Join Zones and AGEBs

In [114]:
# perform spatial join based on whether an ageb geometry intersects a zone geometry
# (i.e. if the ageb geometry if they have any boundary or interior in common with the zone geometry)
stats_census_ageb = gpd.sjoin(stats_census_ageb, primary_zones, how='inner', op='intersects')

In [115]:
# drop the column that is created withen perfoming the join
stats_census_ageb.drop(columns=['index_right'], inplace=True)
# drop columns we don't need
stats_census_ageb.drop(columns=['SHAPE_area', 'HA'], inplace=True)

## Save stats and census data to shapefiles

In [116]:
# rename the columns for saving to shapefiles
rename_stats_columns(stats_census_ageb, inplace=True)

In [117]:
save_shapefiles(stats_census_ageb, 'merida_ageb_stats_census')